# 5.7使用重读元素的网络

深度网络的设计思路：

+ AlexNet

+ VGG,重复使用简单的基础块来构建网络，

...

VGG块的组成：

+ 填充为1，窗口为3\*3的卷积层，

+ 步幅为2，窗口为2\*2的池化层，

+ 卷积层之后，使用ReLU激活函数。

对于给定的感受野（与输出有关的输入图片的局部大小），采用堆积的小卷积核优于采用大的卷积核，因为可以增加网络深度来保证学习更复杂的模式，而且代价还比较小（参数更少）。


In [1]:
import time
import torch 
from torch import nn,optim
import d2l_pytorch as d2l 

import sys
sys.path.append('..')

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# 开始定义VGG网络的基础块
# 该基础块包含：多个卷积层，一个最大池化层。
def vgg_block(num_convs,in_channels,out_channels):
    blk=[]
    for i in range(num_convs):
        if i==0:
            blk.append(nn.Conv2d(in_channels,out_channels,kernel_size=3,padding=1))
        else:
            blk.append(nn.Conv2d(out_channels,out_channels,kernel_size=3,padding=1))
        blk.append(nn.ReLU())
    blk.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*blk)

In [3]:
# 定义VGG网络的全连接层

def vgg_fc(fc_features,fc_hiddens):
    fc=nn.Sequential(d2l.FlattenLayer(),
        nn.Linear(fc_features,fc_hiddens),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hiddens,fc_hiddens),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(fc_hiddens,10)
    )
    return fc

In [4]:
# 开始定义Vgg网络
# 包括：5个卷积块，前2个使用单卷积层，后面3个使用双层卷积层。
# VGG使用了8个卷积层和3个全连接层，所i又叫做VGG-11

# vgg各个卷积块的超参数
conv_arch=(
    (1,  1, 64),
    (1, 64,128),
    (2,128,256),
    (2,256,512),
    (2,512,512)
)
# 使用的数据集是fashion-mnist，所以输入通道数为1.

# VGG全连接层的参数
fc_features=512*7*7     # c*w*h

# VGG隐藏层的参数
fc_hiddens=4096    # 任何？？

In [5]:
# 开始实现VGG网络

def VGG(conv_arch,fc_features,fc_hiddens=4096):
    net=nn.Sequential()
    # 卷积层部分
    for i,(num_convs,in_channels,out_channels) in enumerate(conv_arch):
        net.add_module('vgg_block_'+str(i+1),vgg_block(num_convs,in_channels,out_channels))
    # 全连接层部分
    net.add_module('fc',vgg_fc(fc_features,fc_hiddens))
    return net

In [6]:
# test
net = VGG(conv_arch, fc_features, fc_hiddens)

X = torch.rand(1, 1, 224, 224)

# named_children获取一级子模块及其名字(named_modules会返回所有子模块,包括子模块的子模块)
for name, blk in net.named_children(): 
    X = blk(X)
    print(name, 'output shape: ', X.shape)


vgg_block_1 output shape:  torch.Size([1, 64, 112, 112])
vgg_block_2 output shape:  torch.Size([1, 128, 56, 56])
vgg_block_3 output shape:  torch.Size([1, 256, 28, 28])
vgg_block_4 output shape:  torch.Size([1, 512, 14, 14])
vgg_block_5 output shape:  torch.Size([1, 512, 7, 7])
fc output shape:  torch.Size([1, 10])


可以看到，每次我们将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，所以每层的模型参数尺寸和计算复杂度与输入高、输入宽、输入通道数和输出通道数的乘积成正比。VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

In [7]:
# 获取数据，训练模型

# 够高一个通道数更小的网络，在fashion_mnist数据集上进行训练

ratio = 8
small_conv_arch = [(1, 1, 64//ratio), (1, 64//ratio, 128//ratio), (2, 128//ratio, 256//ratio), 
                   (2, 256//ratio, 512//ratio), (2, 512//ratio, 512//ratio)]
net = VGG(small_conv_arch, fc_features // ratio, fc_hiddens // ratio)
print(net)


Sequential(
  (vgg_block_1): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_2): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (vgg_block_4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
# 加载数据
batch_size = 64 # 将数据分为64个批量，每一次完整的epoch，训练一个批量的数据。
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = d2l.load_data_fashion_mnist_ch05(batch_size, resize=224)

In [9]:
# 开始训练

lr, num_epochs = 0.001, 5

optimizer = torch.optim.Adam(net.parameters(), lr=lr)

d2l.train_ch05_6(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)


er 372/64, loss 0.475
epoch 0/5, iter 373/64, loss 0.568
epoch 0/5, iter 374/64, loss 0.604
epoch 0/5, iter 375/64, loss 0.483
epoch 0/5, iter 376/64, loss 0.303
epoch 0/5, iter 377/64, loss 0.572
epoch 0/5, iter 378/64, loss 0.393
epoch 0/5, iter 379/64, loss 0.409
epoch 0/5, iter 380/64, loss 0.570
epoch 0/5, iter 381/64, loss 0.549
epoch 0/5, iter 382/64, loss 0.517
epoch 0/5, iter 383/64, loss 0.465
epoch 0/5, iter 384/64, loss 0.521
epoch 0/5, iter 385/64, loss 0.628
epoch 0/5, iter 386/64, loss 0.517
epoch 0/5, iter 387/64, loss 0.524
epoch 0/5, iter 388/64, loss 0.578
epoch 0/5, iter 389/64, loss 0.373
epoch 0/5, iter 390/64, loss 0.447
epoch 0/5, iter 391/64, loss 0.406
epoch 0/5, iter 392/64, loss 0.526
epoch 0/5, iter 393/64, loss 0.421
epoch 0/5, iter 394/64, loss 0.476
epoch 0/5, iter 395/64, loss 0.527
epoch 0/5, iter 396/64, loss 0.539
epoch 0/5, iter 397/64, loss 0.266
epoch 0/5, iter 398/64, loss 0.470
epoch 0/5, iter 399/64, loss 0.360
epoch 0/5, iter 400/64, loss 0.46

VGG-11通过5个可以重复使用的卷积块来构造网络。

根据每块里卷积层个数和输出通道数的不同可以定义出不同的VGG模型。
